In [1]:
import glob
import os
import numpy as np
from PIL import Image
import skimage
import skimage.io
import scipy.misc
import imageio
import warnings; warnings.simplefilter('ignore')

list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

list_dict = {}
for i in range(10):
    key = list_classes[i]
    v = [0,0,0,0,0,0,0,0,0,0]
    v[i] = 1
    list_dict[key] = v
print(list_dict)

train_dir = '../input/crop_train'
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
train_files = sorted(glob.glob(train_dir+'/*/*'))
train_data_cnt = len(train_files)
from sklearn.utils import shuffle
train_files = shuffle(train_files,random_state=42)
train_cnt = int(train_data_cnt * 0.8)
valid_cnt = train_data_cnt - train_cnt
BATCH_SIZE = 13
CROP_LEN = 224

def random_crop(im_array):
    # crop
    x_range = im_array.shape[0] - CROP_LEN
    y_range = im_array.shape[1] - CROP_LEN
    # print(x_range,y_range)
    a = np.random.randint(x_range)
    b = a + CROP_LEN
    c = np.random.randint(y_range)
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array

def center_crop(im_array):
    center_x = im_array.shape[0] // 2
    center_y = im_array.shape[1] // 2
    half_crop = CROP_LEN // 2
    a = center_x - half_crop
    b = a + CROP_LEN
    c = center_y - half_crop
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array
    

def get_img(img_path, train_flag = True):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    
    # train or valid
    if train_flag is True:
        final_img = random_crop(im_array)
    else:
        final_img = center_crop(im_array)
    
    final_img = final_img/127.5
    final_img = final_img - 1.0
    return final_img

# 读取测试数据中间 224
def get_test_img(img_path):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    final_img = center_crop(im_array)
    final_img = final_img/127.5
    final_img = final_img - 1.0
    return final_img

def get_y(img_path):
    n = img_path.split('/')[-2]
    return list_dict[n]

def data_gen(file_list, batch_size=BATCH_SIZE, train_flag = True):
    curr_idx = 0
    data_cnt = len(file_list)
    while True:
        if curr_idx + batch_size > data_cnt:
            start_idx = data_cnt-batch_size
            end_idx = data_cnt
            curr_idx = 0
        else:
            start_idx = curr_idx
            end_idx = curr_idx + batch_size
            curr_idx += batch_size
        curr_fl = file_list[start_idx:end_idx]
        curr_x = [get_img(p,train_flag) for p in curr_fl]
        curr_x = np.array(curr_x,dtype='float32')
        curr_y = np.array([get_y(p) for p in curr_fl])
        yield curr_x,curr_y

train_gen = data_gen(train_files[:train_cnt], BATCH_SIZE, True)
valid_gen = data_gen(train_files[train_cnt:], BATCH_SIZE, False)
import math
train_step = math.ceil(train_cnt*0.5/BATCH_SIZE)
valid_step = math.ceil(valid_cnt/BATCH_SIZE)
    

{'Samsung-Galaxy-Note3': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 'LG-Nexus-5x': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'iPhone-6': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'Sony-NEX-7': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Samsung-Galaxy-S4': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'Motorola-X': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'iPhone-4s': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'HTC-1-M7': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'Motorola-Droid-Maxx': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 'Motorola-Nexus-6': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}


In [2]:
# def model
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [3]:
model_p = 'best_resnet_crop.h5'

if os.path.exists(model_p):
    model = load_model(model_p)
    #K.set_value(model.optimizer.lr, 0.0001)
    print('load',model_p)
else:
    from keras.applications.resnet50 import ResNet50
    base_model = ResNet50(input_shape=(CROP_LEN,CROP_LEN,3),include_top=False,weights=None,pooling='max')
    x = base_model.output
    x = Dense(256,activation='relu')(x)
    pred = Dense(10,activation='softmax')(x)
    model = Model(inputs=base_model.input,outputs=pred)
    print('get model')
    print(model.summary())
    model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])

load best_resnet_crop.h5


In [4]:
model_chk = ModelCheckpoint(filepath=model_p, monitor='val_acc', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, verbose=1,
                              patience=3, min_lr=0.00000001)

model.fit_generator(train_gen,
          steps_per_epoch = train_step,
          epochs=60,
          validation_data = valid_gen,
          validation_steps = valid_step,
          callbacks=[model_chk,reduce_lr]
         )

# Epoch 00092: val_acc improved from 0.88524 to 0.89842, saving model to best_resnet_crop.h5
# 1354/1354 [==============================] - 884s 
# 653ms/step - loss: 0.1565 - acc: 0.9449 - val_loss: 0.3287 - val_acc: 0.8984
                

Epoch 1/60
1354/1354 [==============================] - 893s 660ms/step - loss: 0.1372 - acc: 0.9531 - val_loss: 0.4354 - val_acc: 0.8769
Epoch 2/60
1354/1354 [==============================] - 882s 651ms/step - loss: 0.1497 - acc: 0.9498 - val_loss: 0.4378 - val_acc: 0.8752
Epoch 3/60
1354/1354 [==============================] - 883s 652ms/step - loss: 0.1372 - acc: 0.9538 - val_loss: 0.3756 - val_acc: 0.8915
Epoch 4/60
1354/1354 [==============================] - 883s 652ms/step - loss: 0.1453 - acc: 0.9498 - val_loss: 0.3461 - val_acc: 0.8977
Epoch 5/60
1353/1354 [============================>.] - ETA: 0s - loss: 0.1422 - acc: 0.9520Epoch 00005: val_acc did not improve

Epoch 00005: reducing learning rate to 2.499999936844688e-05.
1354/1354 [==============================] - 883s 652ms/step - loss: 0.1421 - acc: 0.9521 - val_loss: 0.5652 - val_acc: 0.8400
Epoch 6/60
1354/1354 [==============================] - 882s 652ms/step - loss: 0.1193 - acc: 0.9618 - val_loss: 0.4600 - val_acc

1354/1354 [==============================] - 880s 650ms/step - loss: 0.0603 - acc: 0.9798 - val_loss: 0.2620 - val_acc: 0.9282
Epoch 31/60
1354/1354 [==============================] - 880s 650ms/step - loss: 0.0586 - acc: 0.9809 - val_loss: 0.3722 - val_acc: 0.9035
Epoch 32/60
1354/1354 [==============================] - 881s 650ms/step - loss: 0.0564 - acc: 0.9822 - val_loss: 0.2827 - val_acc: 0.9269
Epoch 33/60
1354/1354 [==============================] - 880s 650ms/step - loss: 0.0592 - acc: 0.9807 - val_loss: 0.3038 - val_acc: 0.9236
Epoch 34/60
1354/1354 [==============================] - 880s 650ms/step - loss: 0.0556 - acc: 0.9810 - val_loss: 0.2693 - val_acc: 0.9298
Epoch 35/60
1354/1354 [==============================] - 880s 650ms/step - loss: 0.0576 - acc: 0.9807 - val_loss: 0.2925 - val_acc: 0.9263
Epoch 36/60
1354/1354 [==============================] - 880s 650ms/step - loss: 0.0588 - acc: 0.9809 - val_loss: 0.2703 - val_acc: 0.9306
Epoch 37/60
1354/1354 [================

In [5]:
best_model = load_model(model_p)
test_y = []
for img_p in test_files:
    tmp_x = get_test_img(img_p)
    tmp_y = best_model.predict(np.array([tmp_x]))[0]
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

[[  4.95690822e-10   1.35712570e-03   5.03167108e-08   3.10969446e-03
    1.08761049e-03   2.06841531e-08   8.60702188e-04   9.93148744e-01
    4.40775494e-07   4.35597991e-04]
 [  1.31894582e-12   1.28995581e-09   1.12423898e-07   2.89413110e-05
    1.98374025e-07   9.99712646e-01   2.00872591e-06   3.49607898e-09
    1.08468923e-08   2.55942025e-04]
 [  1.02780084e-03   7.05446102e-09   8.94968707e-06   4.10328958e-11
    5.09730178e-08   6.98557514e-08   9.98858094e-01   8.30362260e-05
    7.14190946e-08   2.19450958e-05]
 [  1.18031224e-10   1.55000598e-05   2.37870040e-06   2.82170859e-05
    1.64574736e-08   1.17564687e-08   4.98017243e-07   4.87251928e-05
    9.99701798e-01   2.02797950e-04]
 [  1.89537581e-18   9.99991536e-01   1.24827058e-11   7.59331579e-06
    4.83203699e-09   1.43185797e-09   4.01544978e-11   4.79705664e-10
    8.20581079e-07   6.89134722e-11]]


In [6]:
import pandas as pd
y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
print(df.head())
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/resnet_manip_crop.csv',index=False)

                   fname    camera
0  img_0002a04_manip.tif  iPhone-6
1  img_001e31c_unalt.tif  iPhone-6
2  img_00275cf_manip.tif  iPhone-6
3  img_0034113_unalt.tif  iPhone-6
4  img_00344b7_unalt.tif  iPhone-6
                   fname             camera
0  img_0002a04_manip.tif        LG-Nexus-5x
1  img_001e31c_unalt.tif          iPhone-4s
2  img_00275cf_manip.tif           iPhone-6
3  img_0034113_unalt.tif  Samsung-Galaxy-S4
4  img_00344b7_unalt.tif         Motorola-X


In [7]:
# TTA
def get_tta_img(img_path):
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    im_array = im_array/127.5
    im_array = im_array - 1.0
    img_list = []
    img_list.append(center_crop(im_array))
#     for i in range(4):
#         for j in range(3):
#             a = i*72
#             b = a + 224
#             c = j*96
#             d = c + 224
#             #print(a,b,c,d)
#             img_list.append(im_array[a:b,c:d,:])
    img_list.append(im_array[0:CROP_LEN,0:CROP_LEN,:])
    img_list.append(im_array[0:CROP_LEN,512-CROP_LEN:512,:])
    img_list.append(im_array[512-CROP_LEN:512,0:CROP_LEN,:])
    img_list.append(im_array[512-CROP_LEN:512,512-CROP_LEN:512,:])
    return np.array(img_list)
tta_img = get_tta_img(test_files[0])
print(tta_img.shape)
tta_res = best_model.predict(tta_img)
print(tta_res.shape)
print(np.sum(tta_res,axis=0))

test_y = []
for img_p in test_files:
    tmp_x = get_tta_img(img_p)
    tmp_y = best_model.predict(tmp_x)
    tmp_y = np.sum(tmp_y,axis=0)
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/resnet_manip_crop_tta.csv',index=False)  # score 804

(5, 224, 224, 3)
(5, 10)
[  1.87444730e-05   6.01468608e-03   7.86853582e-02   1.20262019e-02
   5.61773330e-02   3.00654847e-05   7.20329909e-03   4.83296347e+00
   4.75997850e-03   2.12072022e-03]
[[  1.87444730e-05   6.01468608e-03   7.86853582e-02   1.20262019e-02
    5.61773330e-02   3.00654847e-05   7.20329909e-03   4.83296347e+00
    4.75997850e-03   2.12072022e-03]
 [  2.79381857e-06   3.18503469e-01   3.41931544e-02   7.53534187e-05
    1.29285842e-01   4.47995043e+00   1.08112639e-03   5.43277274e-05
    3.80140234e-04   3.64731401e-02]
 [  2.12826245e-02   5.70476279e-02   4.83377837e-03   3.26004374e-05
    3.05653375e-04   1.01136757e-04   4.16344023e+00   7.52126575e-01
    1.08164486e-05   8.19124631e-04]
 [  3.82197030e-08   4.04749345e-03   1.05198487e-05   5.51818346e-04
    8.85308339e-07   2.89436116e-06   1.34401453e-05   7.45816797e-05
    4.99478960e+00   5.08665922e-04]
 [  7.43690332e-10   4.99917555e+00   1.40806005e-04   2.57210068e-05
    6.78630499e-07   1.